In [ ]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.utils import class_weight
import numpy as np
from tensorflow.keras import layers, models

: 

In [ ]:
image_dir = "Data/images"

data = pd.read_csv("Data/metadata.csv")
data_uni = data.drop_duplicates()
print(data_uni.size)


data_uni["filename"] = data_uni["isic_id"] + ".jpg"

data_uni["label"] = data_uni["benign_malignant"]

data_uni["label_index"] = data_uni["label"].astype("category").cat.codes
label_names = data_uni["label"].astype("category").cat.categories.tolist()

print(f"Diagnosis {label_names}")
print(data_uni["label"].head(20))


In [ ]:

data_uni = data_uni.loc[
    (data_uni["benign_malignant"] == "benign") | 
    (data_uni["benign_malignant"] == "malignant"),
    "label_index"
]
data_uni["filepath"] = data_uni["filename"].apply(lambda fname: os.path.join(image_dir, fname))
data_uni = data_uni[data_uni["filepath"].apply(os.path.exists)]

filepaths = data_uni["filepath"].tolist()
labels = data_uni["label_index"].tolist()
# Crear dataset
def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0
    return image, tf.cast(label, tf.int32)

dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
dataset = dataset.map(load_image).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

val_size = int(0.2 * len(filepaths))
val_ds = dataset.take(val_size)
train_ds = dataset.skip(val_size)

In [ ]:
# Calcular pesos según la frecuencia
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(data_uni["label_index"]),
    y=data_uni["label_index"]
)
class_weights = dict(enumerate(class_weights))
print("Pesos para clases:", class_weights)


model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()



In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    class_weight=class_weights
)